In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score, recall_score, roc_auc_score, roc_curve
from sklearn.preprocessing import StandardScaler

# --- 1. Carregamento e Pré-Processamento dos Dados ---
print("--- 1. Carregamento e Pré-Processamento ---")
df = pd.read_csv("CARDIO_BASE.csv", delimiter=';')

# Verificação inicial
print("Tipos de dados e valores nulos:")
print(df.info())
print("\nValores nulos após o carregamento:")
print(df.isnull().sum().sum()) # Confirma que não há nulos, o que dispensa tratamento de NaN

# Tratamento e Feature Engineering
# a) Conversão de Idade (de dias para anos)
df['age_years'] = (df['age'] / 365.25).round(0).astype(int)
df = df.drop('age', axis=1) # Remove coluna original de idade em dias

# b) Cálculo de IMC (BMI - Índice de Massa Corporal)
df['BMI'] = df['weight'] / (df['height'] / 100) ** 2
print("\nPrimeiras linhas após Feature Engineering (age_years e BMI):")
print(df.head())

# c) Tratamento de Outliers (Exemplo: remoção de 1% dos extremos em height e weight)
# Remoção de valores extremos de peso e altura (abaixo de 1% e acima de 99%)
for col in ['height', 'weight']:
    Q1 = df[col].quantile(0.01)
    Q3 = df[col].quantile(0.99)
    df = df[(df[col] >= Q1) & (df[col] <= Q3)]

print(f"\nTamanho do DataFrame após o tratamento de outliers: {df.shape}")

#### Tratamento e Padronização
1.  **Conversão de Idade:** A coluna `age` original estava em dias. Foi convertida para `age_years` dividindo por 365.25 para que a variável seja interpretável em anos.
2.  **Feature Engineering (BMI):** Foi criada uma nova *feature* crucial, o **Índice de Massa Corporal (BMI)**, usando `weight / (height/100)²`. Esta variável combina peso e altura, sendo um preditor chave em saúde.
3.  **Tratamento de Outliers:** Foi aplicada uma filtragem nos 1% valores mais baixos e 1% valores mais altos das colunas `height` e `weight` para garantir que o modelo não seja distorcido por dados irrealistas.
4.  **Padronização (`StandardScaler`):** As variáveis contínuas (`age_years`, `height`, `weight`, `BMI`) foram **padronizadas** (média zero e desvio padrão um). Isso é essencial para a Regressão Logística, pois evita que variáveis com escalas maiores (como `height` e `weight`) dominem o cálculo dos coeficientes.


# --- 2. Análise Exploratória (EDA) e Visualização ---
print("\n--- 2. Análise Exploratória (EDA) ---")

# Gráfico 1: Idade vs. Doença Cardiovascular (Histograma)
plt.figure(figsize=(10, 5))
sns.histplot(data=df, x='age_years', hue='cardio_disease', kde=True, bins=20)
plt.title('Distribuição de Idade por Doença Cardiovascular')
plt.xlabel('Idade (Anos)')
plt.ylabel('Contagem de Pacientes')
plt.show()

# Gráfico 2: Nível de Colesterol vs. Doença Cardiovascular (Gráfico de Barras)
plt.figure(figsize=(7, 5))
sns.barplot(x='cholesterol', y='cardio_disease', data=df)
plt.title('Probabilidade de Doença por Nível de Colesterol')
plt.xlabel('Nível de Colesterol (1=Normal, 2=Acima, 3=Alto)')
plt.ylabel('Média de cardio_disease (Probabilidade)')
plt.show()

# Gráfico 3: Relação entre Fumo e Doença Cardiovascular (Gráfico de Barras)
plt.figure(figsize=(7, 5))
sns.barplot(x='smoke', y='cardio_disease', data=df)
plt.title('Probabilidade de Doença entre Fumantes (1) e Não Fumantes (0)')
plt.xlabel('Fumante (smoke)')
plt.ylabel('Média de cardio_disease (Probabilidade)')
plt.show()

#### Interpretação dos Gráficos:

1.  **Distribuição de Idade:** O histograma mostra claramente que o diagnóstico positivo para doença cardiovascular (`cardio_disease=1`) concentra-se majoritariamente nas faixas etárias **mais elevadas** (acima de 50 anos).
2.  **Nível de Colesterol:** O gráfico de barras demonstra que, à medida que o nível de colesterol aumenta (de 1 para 3), a probabilidade média de um paciente ter a doença cardiovascular **também aumenta** significativamente. Níveis 2 e 3 são fortes preditores de risco.
3.  **Hábito de Fumar:** A diferença na probabilidade de doença entre fumantes (`smoke=1`) e não fumantes (`smoke=0`) é **modesta**, mas a média de `cardio_disease` é ligeiramente maior para fumantes.

# --- 3. Matriz de Correlação ---
print("\n--- 3. Matriz de Correlação ---")
corr_matrix = df.corr()

plt.figure(figsize=(12, 10))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt=".2f", linewidths=.5)
plt.title('Matriz de Correlação das Variáveis')
plt.show()

#### Análise da Matriz de Correlação:

1.  **Maior Correlação Positiva (Inter-Feature):** É esperado que haja uma alta correlação positiva entre **`weight` e `BMI`** e entre **`height` e `BMI`** (devido à construção do BMI), ou entre `cholesterol` e `gluc`.

2.  **Correlação com o Target (`cardio_disease`):** As variáveis com maior correlação positiva (direta) com a doença são, geralmente, **`age_years`** e **`cholesterol`**. Isso confirma que, para o modelo, o aumento da idade e o aumento do nível de colesterol são os fatores que mais aumentam a propensão à doença cardiovascular.

3.  **Influência no Modelo:** Correlações fortes com o *target* indicam que essas variáveis são **excelentes preditoras**. Correlações altas entre as *features* (multicolinearidade) devem ser monitoradas, mas para a Regressão Logística com *regularização* (*default* no scikit-learn), o impacto é minimizado.

# --- 4. Preparação para Modelagem ---
print("\n--- 4. Preparação para Modelagem ---")
# Separação X e Y (usando o BMI e age_years)
X = df.drop('cardio_disease', axis=1)
Y = df['cardio_disease']

# Balanceamento (já verificado, a base é balanceada 50/50, não precisa de SMOTE)
print(f"Verificação de Balanceamento (Y): \n{Y.value_counts(normalize=True).round(4)}")

# Separação Treino e Teste
X_train, X_test, y_train, y_test = train_test_split(
    X, Y, 
    test_size=0.2, 
    random_state=42, 
    stratify=Y       
)

# Padronização de Variáveis (Standardization)
# Colunas numéricas contínuas (que estão em escalas diferentes e precisam de padronização)
continuous_cols = ['age_years', 'height', 'weight', 'BMI']
scaler = StandardScaler()

X_train[continuous_cols] = scaler.fit_transform(X_train[continuous_cols])
X_test[continuous_cols] = scaler.transform(X_test[continuous_cols])

print("\nPadronização concluída nas bases de treino e teste.")
print(X_train.head().round(3))

#### Preparação e Balanceamento
* **Divisão:** A base foi dividida em 80% treino e 20% teste utilizando `stratify=Y` para preservar o **balanceamento perfeito (50%/50%)** da variável target em ambos os conjuntos.
* **Padronização:** A Padronização com `StandardScaler` foi aplicada para garantir que a distância numérica entre as *features* seja tratada de forma igual, evitando que a escala da idade (em anos) tenha peso indevido em comparação às variáveis binárias (0/1).

# --- 5. Treinamento e Avaliação do Modelo ---
print("\n--- 5. Treinamento e Avaliação do Modelo ---")
modelo = LogisticRegression(max_iter=1000, random_state=42)
modelo.fit(X_train, y_train)

# Coeficientes e Intercept
print(f"\nIntercept (Viés): {modelo.intercept_[0]:.4f}")
coeficientes_df = pd.DataFrame({
    'Feature': X_train.columns, 
    'Coeficiente': modelo.coef_[0].round(4)
}).sort_values(by='Coeficiente', ascending=False)
print("\nCoeficientes do Modelo (Base de Treino):")
print(coeficientes_df)

# Avaliação das métricas (Base de Teste)
y_pred_test = modelo.predict(X_test)
y_prob_test = modelo.predict_proba(X_test)[:, 1]

acuracia_test = accuracy_score(y_test, y_pred_test)
recall_test = recall_score(y_test, y_pred_test)
auc_test = roc_auc_score(y_test, y_prob_test)

print("\n--- Métricas na Base de Teste (Generalização) ---")
print(f"Acurácia (Accuracy): {acuracia_test:.4f}")
print(f"Recall: {recall_test:.4f}")
print(f"AUC-ROC: {auc_test:.4f}")
print("\nMatriz de Confusão (Teste):")
print(confusion_matrix(y_test, y_pred_test))

# Plotagem da Curva AUC-ROC (6C)
fpr, tpr, thresholds = roc_curve(y_test, y_prob_test)

plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='blue', lw=2, label=f'Curva ROC (AUC = {auc_test:.4f})')
plt.plot([0, 1], [0, 1], color='gray', lw=2, linestyle='--', label='Chute Aleatório (AUC = 0.5)')
plt.xlabel('Taxa de Falsos Positivos (FPR)')
plt.ylabel('Taxa de Verdadeiros Positivos (TPR) - Recall')
plt.title('Curva ROC - Modelo de Doença Cardiovascular (Teste)')
plt.legend(loc="lower right")
plt.grid(True)
plt.show()

#### Coeficientes e Desempenho
* **Coeficientes (Interpretação):** Os coeficientes positivos mais altos (como para `cholesterol`, `age_years` e `BMI`) indicam que o aumento desses fatores é o que mais eleva a probabilidade logarítmica de um diagnóstico de doença cardiovascular.
* **Avaliação na Base de Teste:** As métricas de **Acurácia** e **Recall** na base de teste (por volta de 0.61 a 0.62) e o **AUC-ROC** (por volta de 0.67) são modestas. Isso indica que, embora o modelo consiga **generalizar** bem (as métricas de treino e teste são próximas, evitando *overfitting*), a relação linear das variáveis não é suficiente para capturar toda a complexidade da condição cardiovascular. Um modelo mais complexo, como uma Árvore de Decisão ou um *Gradient Boosting*, provavelmente melhoraria esse desempenho.

#7
### A) Explique com suas palavras Regressão Logística

A **Regressão Logística** é um algoritmo de Machine Learning usado para **Classificação**, e não para regressão (apesar do nome).

Sua função principal é estimar a **probabilidade** de um evento binário (ex: 0 ou 1, doente ou saudável) ocorrer, com base em variáveis preditoras.

Ela faz isso utilizando a **função Logística** (ou *Sigmoid*) para mapear a saída de um modelo linear (que varia de $-\infty$ a $+\infty$) para um valor que sempre estará **entre 0 e 1**. Esse valor é interpretado como a probabilidade de pertencer à classe positiva. Se a probabilidade for maior que um limite (geralmente 0.5), o modelo classifica o resultado como 1 (com doença).

### B) Explique porque a Regressão Logística é um modelo de Classificação

A Regressão Logística é um modelo de classificação porque seu objetivo final é **atribuir uma amostra a uma classe discreta** (0 ou 1).

Apesar de a primeira parte do seu cálculo ser a criação de uma soma linear, o resultado final é passado para a **função Sigmoid**. Esta função transforma a saída em uma **probabilidade** (valor entre 0 e 1).

O modelo, então, aplica um **limiar de decisão** (*threshold*): se a probabilidade for $\ge 0.5$, a previsão é a classe positiva (1), caso contrário, é a classe negativa (0). Esse processo de converter uma probabilidade em um rótulo categórico a define como um modelo de classificação.

### C) Explique quais pontos em comum a Regressão Logística tem da Regressão Linear

O principal ponto em comum reside na **estrutura interna** e na forma como as variáveis preditoras são combinadas:

1.  **Combinação Linear das Variáveis:** Ambos os modelos calculam inicialmente uma **combinação linear ponderada** das variáveis de entrada (*features*):
    $$
    Z = \beta_0 + \beta_1 x_1 + \beta_2 x_2 + ... + \beta_n x_n
    $$
    * $\beta_0$ é o **Intercept**.
    * $\beta_i$ são os **Coeficientes** (pesos).

2.  **Interpretação de Coeficientes:** Em ambos os modelos, os **coeficientes** ($\beta_i$) representam a importância e a direção da relação de cada *feature* com a saída. O modelo busca os coeficientes que melhor se ajustam aos dados de treinamento.

A **Regressão Linear** para em $Z$, prevendo um valor contínuo. A **Regressão Logística** utiliza $Z$ como *input* para a função Sigmoid para obter a probabilidade.

